In [ ]:
import os
import itertools
import subprocess

# Set up file paths
folder_with_pdbs = "./input_pdbs"
output_dir = "./output_pdbs"
path_for_parsed_chains = os.path.join(output_dir, "parsed_pdbs.jsonl")
path_for_assigned_chains = os.path.join(output_dir, "assigned_pdbs.jsonl")
chains_to_design = "A"

# Create output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Run initial parsing scripts (unchanged for all iterations)
subprocess.run([
    "python", "../helper_scripts/parse_multiple_chains.py",
    "--input_path", folder_with_pdbs,
    "--output_path", path_for_parsed_chains
])

subprocess.run([
    "python", "../helper_scripts/assign_fixed_chains.py",
    "--input_path", path_for_parsed_chains,
    "--output_path", path_for_assigned_chains,
    "--chain_list", chains_to_design
])

# Define parameter values for iteration
sampling_temps = [0.1, 0.2, 0.5]  # Example values
backbone_noises = [0.01, 0.02]    # Example values
seeds = [37, 42]                  # Example values
use_soluble_model_values = [True, False]

# Iterate over all combinations of parameters
for sampling_temp, backbone_noise, seed, use_soluble_model in itertools.product(sampling_temps, backbone_noises, seeds, use_soluble_model_values):
    
    # Construct the command
    cmd = [
        "python", "../protein_mpnn_run.py",
        "--jsonl_path", path_for_parsed_chains,
        "--chain_id_jsonl", path_for_assigned_chains,
        "--out_folder", output_dir,
        "--num_seq_per_target", "2",
        "--sampling_temp", str(sampling_temp),
        "--seed", str(seed),
        "--batch_size", "1",
        "--backbone_noise", str(backbone_noise)
    ]
    
    # Add use_soluble_model flag conditionally
    if use_soluble_model:
        cmd.append("--use_soluble_model")
    
    # Print the command for logging purposes (optional)
    print(f"Running with sampling_temp={sampling_temp}, backbone_noise={backbone_noise}, seed={seed}, use_soluble_model={use_soluble_model}")
    
    # Execute the command
    subprocess.run(cmd)
